In [39]:

import numpy as np
import pandas as pd 
import decimal
import json

# pathFolder = '/Users/dmitriytrofimov/DEV/dmitryitrofimov.github.io/json/thread/m/'
pathFolder = 'e:/DEV/FLUTTER/project/MY_GITHUB/dmitryitrofimov/json/thread/m/'

In [40]:
def read_excel(path,sheet_name):
    df = pd.read_excel(io = path,sheet_name =sheet_name,  header = 0,index_col=None,)

   
    # name_column = df.columns.tolist()
    # for s in range(len(name_column)):
    # # del space start
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.lstrip()
    # # del space end
    #      df[name_column[s]] = df[name_column[s]].astype(str).str.rstrip()
    return df

 
def to_json(df, nameFile):
    df.to_json(path_or_buf =pathFolder+nameFile.lower()+'.json' , orient="records") 
    # df.to_json(path_or_buf ='e:/DEV/FLUTTER/project/MY_GITHUB/dmitryitrofimov/json/thread/m/'+nameFile.lower()+'.json' , orient="records") 
    # df.to_json(path_or_buf ='../json/m/'+nameFile.lower()+'.json' , orient="records") 
def to_json_from_list (list, nameFile):
    with open(pathFolder+nameFile.lower()+'.json', 'w') as f:
        json.dump(list, f)
    
    
def get_patch_json(file):
    return pathFolder+file.lower()+'.json' 

def get_patch_from(file):
    return '../excel/'+file+'.xlsx'

def change_types(df,columnType):
    for k, v in columnType.items(): 
        df[k] = df[k].astype(v)


def to_list_float(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [float(x) for x in df[nameCol].values[index].split(',')]

def to_list_String(df,nameCol)   :
    for index, row in df.iterrows():
        value = df[nameCol].values[index]
        if(str(value).lower() != 'nan'):
            df[nameCol].values[index] = [str(x) for x in df[nameCol].values[index].split(',')]
            
            
def normalize_fraction(d):
    normalized = d.normalize()
    sign, digits, exponent = normalized.as_tuple()
    if exponent > 0:
        return decimal.Decimal((sign, digits + (0,) * exponent, 0))
    else:
        return normalized

def toStringWithoutZero(s):
    return s.rstrip('0').rstrip('.') if '.' in s else s        
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]

# m_thread

In [41]:
file_excel = 'm_thread'

sheet_names = [
    'm_diam',
    'm_profile',
    'm_tolerance',
 
               ]

# ******************************
patchFrom = get_patch_from(file_excel)


df_base = read_excel(patchFrom, sheet_names[0])





In [42]:
df_m_diam = read_excel(patchFrom, sheet_names[0])
df_m_profile = read_excel(patchFrom, sheet_names[1])
df_m_tolerance = read_excel(patchFrom, sheet_names[2])

In [43]:
# column
diam = 'diam'
diamFormat = 'desc'
coarsePitch = 'coarsePitch'
finePitch = 'finePitch'
extraFinePitch = 'extraFinePitch'
rangeMain = 'rangeMain'
rangeSub = 'rangeSub'

# create df from df base

In [44]:
df_update = df_base[diam].unique()
df_update = pd.DataFrame( df_update, columns=[diam])

In [45]:
"""     required String desc,
    required String? coarsePitch,
    required List<String>? finePitch,
    required List<String>? extraFinePitch,
    required int? rangeMain,
    required int? rangeSub,
 """

'     required String desc,\n    required String? coarsePitch,\n    required List<String>? finePitch,\n    required List<String>? extraFinePitch,\n    required int? rangeMain,\n    required int? rangeSub,\n '

In [46]:
diam = 0
pitch = 0
for i, row in df_update.iterrows():
    diam = df_update.diam[i]
   
    
    
    df_update.at[i, 'desc']='M '+ toStringWithoutZero(str(diam))
    
    find_row =df_base.query("diam == @diam")
    if (find_row.count().sum())>0:
        range_main =  find_row.iloc[0]['range_main']
        range_sub =  find_row.iloc[0]['range_sub']
        
        df_update.at[i, rangeMain]=range_main
        df_update.at[i, rangeSub]=range_sub
    
    
    # coarse pitch
    find_row = df_base.query("diam == @diam and type_pitch == 1")
    if (find_row.count().sum())>0:
        pitch =  find_row.iloc[0]['pitch']
        df_update.at[i, coarsePitch]=toStringWithoutZero(str(pitch))
   
    # fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 2")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, finePitch]=str(pitch).replace('[','').replace(']','')  
    
    # extra fine pitch
    find_row = df_base.query("diam == @diam and type_pitch == 3")
    if (find_row.count().sum())>0:
        pitch =  find_row['pitch'].tolist()
        df_update.at[i, extraFinePitch]=str(pitch).replace('[','').replace(']','')  




# записываем диаметры

In [47]:
to_list_String(df_update,finePitch)
to_list_String(df_update,extraFinePitch)

df_update = df_update.drop(df_update.columns[[0]], axis=1)
to_json(df_update,'diam/diam')
    

In [48]:
coarsePitch = 'coarsePitch'

In [49]:
# df_pitch = pd.DataFrame(columns=[diamPitch])

for i, row in df_update.iterrows():
    df_pitch = pd.DataFrame()
   
    
    diamFormat = df_update.desc[i]
    coarse = df_update.coarsePitch[i]
    fine = df_update.finePitch[i]
    extra = df_update.extraFinePitch[i]
   
   
    # if not np.isnan(coarse):
  
    if str(coarse)!='nan':
        df_pitch.at[i, coarsePitch]=diamFormat +' x ' + toStringWithoutZero(str(coarse))
   
   
    #  отсеиваю все с nan
    if str(fine)!='nan':

        list2 = []
        for item in fine:
            s = str(item).replace('[','').replace(']','').replace(' ','')
            list2.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, finePitch]=str(list2)
        
    if str(extra)!='nan':

        list2 = []
        for item in extra:

            s = str(item).replace('[','').replace(']','').replace(' ','')
            list2.append(diamFormat+ ' x ' +  toStringWithoutZero(s))
         
        
        df_pitch.at[i, extraFinePitch]=str(list2)        
    
    
    # df_pitch[finePitch]= df_pitch[finePitch].astype('object')
    # to_list_float(df_pitch,finePitch)
    nameFile = diamFormat.replace(' ','')

    to_json(df_pitch, 'pitch/'+nameFile)
    
    # Read in the file
    with open(get_patch_json('pitch/'+nameFile), 'r') as file :
        filedata = file.read()

# Replace the target string
        filedata = filedata.replace('"[', '[').replace(']"', ']').replace("'", '"')

# Write the file out again
    with open(get_patch_json('pitch/'+nameFile), 'w') as file:
        file.write(filedata)
        
      

# write tolerance

In [50]:
df_tolerance = read_excel(patchFrom, sheet_names[2])
df_diam = read_excel(patchFrom, sheet_names[0])

def toleranceInfo(tolerance):
    return 'M '+ toStringWithoutZero(str(diam))+' x '+ toStringWithoutZero(str(pitch))+ ' - '+ tolerance
def uniq(x):
  return pd.unique(x).tolist()   

In [51]:

diam = 90
pitch = 1

if diam == 600 or diam ==1:
    find_row =df_tolerance.query("diameter_min <= @diam and diameter_max >= @diam and pitch ==@pitch")
else :
    find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
    # print(find_row.shape())


In [52]:
diam = 0.0
pitch = 0.0


for i, row in df_diam.iterrows():
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]


    if diam == 600 or diam ==1:
        find_row =df_tolerance.query("diameter_min <= @diam and diameter_max >= @diam and pitch ==@pitch")
    else :
        find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
        
    # print(find_row.shape())
    if (find_row.count().sum())==0: continue
# delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
# deleted 1 2 3 column
    find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1)
# print(find_row2)
    column_headers = find_row2.columns.values.tolist()
    
    male = []
    female = []

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append(toleranceInfo(itemArray[2]))
        else:
            female.append(toleranceInfo(itemArray[2].upper()))
    # оставляю уникальные
    male = uniq(male)    
    female = uniq(female)    
    
    # df_male = pd.DataFrame(male, columns =['name'])
    # df_female = pd.DataFrame(female, columns =['name'])
    # print(str(diam) + " " + str(pitch))
    
    to_json_from_list(male,'tolerance/male/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))
    to_json_from_list(female,'tolerance/female/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))
    # to_json(df_male,'tolerance/male/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))
    # to_json(df_female,'tolerance/female/M'+ toStringWithoutZero(str(diam))+'x'+ toStringWithoutZero(str(pitch)))

# TEST

In [53]:
# diam = 3
# pitch = 0.35

# tolerance = '_6g_m'


# if diam == 600:
#     find_row =df_tolerance.query("diameter_min <= @diam and diameter_max >= @diam and pitch ==@pitch")
# else :
#     find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
# df_m_diam_find =df_m_diam.query("diam == @diam and pitch ==@pitch")

# df_m_profile_find =df_m_profile.query("pitch == @pitch")

#     # delete column if nan
# find_row2 = find_row.dropna(axis=1, how='all')
#    # deleted 1 2 3 column
# find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,)
# column_headers = find_row2.columns.values.tolist()
# find_row3 = find_row2.filter(regex=tolerance)
# find_row3 = find_row3.reset_index(drop=True)
# find_row3 = find_row3.apply(lambda x: x / 1000)

# isBolt =False
# if tolerance.split('_')[2]=='m': 
#     isBolt = True
# find_row3.at[0, 'isMale']= isBolt

# if isBolt:
#     find_row3.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1] 
# else:
#     find_row3.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1].upper() 
 
# find_row3.at[0, 'diam']=toStringWithoutZero(str(diam))    
# find_row3.at[0, 'pitch']=toStringWithoutZero(str(pitch))    
# find_row3.at[0, 'tolerance']=tolerance.split('_')[1] 

#     #    
# find_row3.at[0, 'pitch_diam_d2']= df_m_diam_find.pitch_diam_d2.iloc[0]
# find_row3.at[0, 'minor_diam_d1']= df_m_diam_find.minor_diam_d1.iloc[0]
# find_row3.at[0, 'minor_diam_d3']= df_m_diam_find.minor_diam_d3.iloc[0]
# find_row3.at[0, 'type_pitch']= df_m_diam_find.type_pitch.iloc[0]
# find_row3.at[0, 'range_main']= df_m_diam_find.range_main.iloc[0]
# find_row3.at[0, 'range_sub']= df_m_diam_find.range_sub.iloc[0]

# find_row3.at[0, 'h']= df_m_profile_find.h.iloc[0]
# find_row3.at[0, 'h_5_8']= df_m_profile_find.h_5_8.iloc[0]
# find_row3.at[0, 'h_3_8']= df_m_profile_find.h_3_8.iloc[0]
# find_row3.at[0, 'h_4']= df_m_profile_find.h_4.iloc[0]
# find_row3.at[0, 'h_8']= df_m_profile_find.h_8.iloc[0]



# find_row3.columns = find_row3.columns.str.replace(tolerance, '')
# if isBolt:
#     find_row3.at[0, 'depth']= (float(find_row3.diam.iloc[0])-find_row3.minor_diam_d3.iloc[0])/2
    
#     find_row3.at[0, 'major_diam_max']= float(find_row3.diam.iloc[0])+find_row3.d_es.iloc[0]
#     find_row3.at[0, 'major_diam_min']= float(find_row3.diam.iloc[0])+find_row3.d_ei.iloc[0]
#     find_row3.at[0, 'major_diam_medium ']= (find_row3.major_diam_min.iloc[0] + find_row3.major_diam_max.iloc[0] )/2
     
# else:
#     find_row3.at[0, 'depth']= (float(find_row3.diam.iloc[0]) -find_row3.minor_diam_d1.iloc[0])/2
    
#     find_row3.at[0, 'minor_diam_max']= find_row3.minor_diam_d1.iloc[0]+find_row3.d1_es.iloc[0]
#     find_row3.at[0, 'minor_diam_min']= find_row3.minor_diam_d1.iloc[0]+find_row3.d1_ei.iloc[0]
#     find_row3.at[0, 'minor_diam_medium']= (find_row3.minor_diam_min.iloc[0] + find_row3.minor_diam_max.iloc[0] )/2

# find_row3.at[0, 'mean_diam_max']= find_row3.pitch_diam_d2.iloc[0]+find_row3.d2_es.iloc[0]
# find_row3.at[0, 'mean_diam_min']= find_row3.pitch_diam_d2.iloc[0]+find_row3.d2_ei.iloc[0]
# find_row3.at[0, 'mean_diam_medium']= (find_row3.mean_diam_max.iloc[0] + find_row3.mean_diam_min.iloc[0] )/2
          
# find_row3 = change_column_order(find_row3,'info',0)
# find_row3 = change_column_order(find_row3,'diam',1)
# find_row3 = change_column_order(find_row3,'pitch',2)
# find_row3 = change_column_order(find_row3,'tolerance',3)
# find_row3 = change_column_order(find_row3,'isMale',4)
# find_row3 = change_column_order(find_row3,'depth',4)
# find_row3 = change_column_order(find_row3,'type_pitch',5)
# find_row3 = change_column_order(find_row3,'range_main',6)
# find_row3 = change_column_order(find_row3,'range_sub',7)
# # 
# find_row3 = change_column_order(find_row3,'mean_diam_max',11)
# find_row3 = change_column_order(find_row3,'mean_diam_medium',12)
# find_row3 = change_column_order(find_row3,'mean_diam_min',13)



In [54]:
diam = 0
for i, row in df_diam.iterrows():
    # что бы всю итерацию не ждать
    # if diam ==1.1: break
    
    diam = df_diam.diam[i]
    pitch = df_diam.pitch[i]

    if diam == 600 or diam ==1:
        find_row =df_tolerance.query("diameter_min <= @diam and diameter_max >= @diam and pitch ==@pitch")
    else :
        find_row =df_tolerance.query("diameter_min < @diam and diameter_max >= @diam and pitch ==@pitch")
    
    df_m_diam_find =df_m_diam.query("diam == @diam and pitch ==@pitch")
    df_m_profile_find =df_m_profile.query("pitch == @pitch")
    
    # если пусто продолжаем
    if (find_row.count().sum())==0: continue
    
    # delete column if nan
    find_row2 = find_row.dropna(axis=1, how='all')
   # deleted 1 2 3 column
    find_row2 = find_row2.drop(find_row2.columns[[0,1,2,3]], axis=1,)
    
    
     

    tolerance = []
    male = []
    female = []
    column_headers = find_row2.columns.values.tolist()

    for item in column_headers:
        itemArray = str(item).split('_')
        if itemArray[3]=='m':
            male.append('_'+itemArray[2]+'_m')
        else:
            female.append('_'+itemArray[2]+'_f')
            
       # оставляю уникальные
    male = uniq(male)    
    female = uniq(female)    
    #  обьединяю допуски
    male.extend(female)
    all = male
    
    isMale =False
    for tolerance in all:
        print( str(diam) + " " + tolerance +' - '+ str(pitch))
        # bolt 
        if tolerance.split('_')[2]=='m': isMale = True
        else: isMale = False
        
        # выборка строки 
        df = find_row2.filter(regex=tolerance)
        df = df.apply(lambda x: x / 1000)
        df = df.reset_index(drop=True)
        df.columns = df.columns.str.replace(tolerance, '')
        
        df.at[0, 'diam']=toStringWithoutZero(str(diam))    
        df.at[0, 'pitch']=toStringWithoutZero(str(pitch))    
        
        df.at[0, 'pitch_diam_d2']= df_m_diam_find.pitch_diam_d2.iloc[0]
        df.at[0, 'minor_diam_d1']= df_m_diam_find.minor_diam_d1.iloc[0]
        df.at[0, 'minor_diam_d3']= df_m_diam_find.minor_diam_d3.iloc[0]
        df.at[0, 'type_pitch']= df_m_diam_find.type_pitch.iloc[0]
        df.at[0, 'range_main']= df_m_diam_find.range_main.iloc[0]
        df.at[0, 'range_sub']= df_m_diam_find.range_sub.iloc[0]

        df.at[0, 'h']= df_m_profile_find.h.iloc[0]
        df.at[0, 'h_5_8']= df_m_profile_find.h_5_8.iloc[0]
        df.at[0, 'h_3_8']= df_m_profile_find.h_3_8.iloc[0]
        df.at[0, 'h_4']= df_m_profile_find.h_4.iloc[0]
        df.at[0, 'h_8']= df_m_profile_find.h_8.iloc[0]
        df.at[0, 'isMale']= isMale
        
        
        
        if isMale:
            df.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1] 
            df.at[0, 'tolerance']=tolerance.split('_')[1] 
            
            df.at[0, 'depth']= (float(df.diam.iloc[0])-df.minor_diam_d3.iloc[0])/2

            df.at[0, 'major_diam_max']= float(df.diam.iloc[0])+df.d_es.iloc[0]
                               
            df.at[0, 'major_diam_min']= float(df.diam.iloc[0])+df.d_ei.iloc[0]
            df.at[0, 'major_diam_mean']= (df.major_diam_min.iloc[0] + df.major_diam_max.iloc[0] )/2

            
        else:
            df.at[0, 'info']= 'M '+ toStringWithoutZero(str(diam))+' x ' + toStringWithoutZero(str(pitch))+' - ' +tolerance.split('_')[1].upper()
            df.at[0, 'tolerance']=tolerance.split('_')[1].upper()
            df.at[0, 'depth']= (float(df.diam.iloc[0]) -df.minor_diam_d1.iloc[0])/2
                
            df.at[0, 'minor_diam_max']= df.minor_diam_d1.iloc[0]+df.d1_es.iloc[0]
            df.at[0, 'minor_diam_min']= df.minor_diam_d1.iloc[0]+df.d1_ei.iloc[0]
            df.at[0, 'minor_diam_mean']= (df.minor_diam_min.iloc[0] + df.minor_diam_max.iloc[0] )/2

            

              
        df.at[0, 'pitch_diam_max']= df.pitch_diam_d2.iloc[0]+df.d2_es.iloc[0]
        df.at[0, 'pitch_diam_min']= df.pitch_diam_d2.iloc[0]+df.d2_ei.iloc[0]
        df.at[0, 'pitch_diam_mean']= (df.pitch_diam_max.iloc[0] + df.pitch_diam_min.iloc[0] )/2  

                
        df = change_column_order(df,'info',0)
        df = change_column_order(df,'diam',1)
        df = change_column_order(df,'pitch',2)
        df = change_column_order(df,'tolerance',3)
        df = change_column_order(df,'isMale',4)
        df = change_column_order(df,'depth',5)
        df = change_column_order(df,'type_pitch',6)
        df = change_column_order(df,'range_main',7)
        df = change_column_order(df,'range_sub',8)
        if isMale:
            df = change_column_order(df,'major_diam_max',9)
            df = change_column_order(df,'major_diam_mean',10)
            df = change_column_order(df,'major_diam_min',11)
        else:
            df = change_column_order(df,'minor_diam_max',9)
            df = change_column_order(df,'minor_diam_mean',10)
            df = change_column_order(df,'minor_diam_min',11)
        
        df = change_column_order(df,'pitch_diam_max',12)
        df = change_column_order(df,'pitch_diam_mean',13)
        df = change_column_order(df,'pitch_diam_min',14)
        
        df = change_column_order(df,'pitch_diam_d2',15)
        df = change_column_order(df,'minor_diam_d1',16)
        df = change_column_order(df,'minor_diam_d3',17)
        
        if isMale:
            to_json(df,'info/male/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])
        else:
            to_json(df,'info/female/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])
            
            
        #  

1.0 _3h4h_m - 0.25
1.0 _4g_m - 0.25
1.0 _4h_m - 0.25
1.0 _5h4h_m - 0.25
1.0 _5g6g_m - 0.25
1.0 _5h6h_m - 0.25
1.0 _6f_m - 0.25
1.0 _6g_m - 0.25
1.0 _6h_m - 0.25
1.0 _7g6g_m - 0.25
1.0 _7h6h_m - 0.25
1.0 _8h6h_m - 0.25
1.0 _4h_f - 0.25
1.0 _4h5h_f - 0.25
1.0 _5g_f - 0.25
1.0 _5h_f - 0.25
1.0 _6g_f - 0.25
1.0 _6h_f - 0.25
1.0 _3h4h_m - 0.2
1.0 _4g_m - 0.2
1.0 _4h_m - 0.2
1.0 _5h4h_m - 0.2
1.0 _5g6g_m - 0.2
1.0 _5h6h_m - 0.2
1.0 _6f_m - 0.2
1.0 _6g_m - 0.2
1.0 _6h_m - 0.2
1.0 _7g6g_m - 0.2
1.0 _7h6h_m - 0.2
1.0 _8h6h_m - 0.2
1.0 _4h_f - 0.2
1.0 _4h5h_f - 0.2
1.0 _5g_f - 0.2
1.0 _5h_f - 0.2
1.0 _6g_f - 0.2
1.0 _6h_f - 0.2
1.1 _3h4h_m - 0.25
1.1 _4g_m - 0.25
1.1 _4h_m - 0.25
1.1 _5h4h_m - 0.25
1.1 _5g6g_m - 0.25
1.1 _5h6h_m - 0.25
1.1 _6f_m - 0.25
1.1 _6g_m - 0.25
1.1 _6h_m - 0.25
1.1 _7g6g_m - 0.25
1.1 _7h6h_m - 0.25
1.1 _8h6h_m - 0.25
1.1 _4h_f - 0.25
1.1 _4h5h_f - 0.25
1.1 _5g_f - 0.25
1.1 _5h_f - 0.25
1.1 _6g_f - 0.25
1.1 _6h_f - 0.25
1.1 _3h4h_m - 0.2
1.1 _4g_m - 0.2
1.1 _4h_m - 0.2
1.

In [ ]:


    #     if isBolt:
    #         to_json(find_row3,'info/male/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])
    #     else:
    #         to_json(find_row3,'info/female/m'+toStringWithoutZero(str(diam))+'x'+toStringWithoutZero(str(pitch))+'-'+tolerance.split('_')[1])
            
            

# print(female)
# print(find_row3)

